<a href="https://colab.research.google.com/github/janmlew/TensorFlow/blob/master/TensorFlow_C3W4_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>